# Manual DQN - Cartpole

This notebook implements the **Deep Q Network (DQN)** for the [CartPole](https://gym.openai.com/envs/CartPole-v1/) game manually.
See `../ReinforcementLearning_Guide.md` for theory and intuition.

According to the OpenAI environment page of CartPole: "A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force to the left (0) or right (1) of the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright (i.e., does not fall). The episode ends when the pole is more than 12 degrees from vertical, or the cart moves more than 2.4 units from the center."

Note that I needed to change the description above, since it is outdated on the web page.

Look at the Github page of the environment, on the docstring of the environment.

Note the following:
- The center position is 0 and the range of possibles postions is `[-2.4,2.4]`, continuous
- Pole angle can vary in `[-12,12] deg`
- Velocity (linear for cart, angular for pole) can be any
- An episode is done if
    1. The pole tresspasses the limits above
    2. 200 steps/actions taken
    3. A minimum return is achieved over 100 steps/actions

We need to discretize the domains using bins.

Overview of sections:



In [1]:
import numpy as np
import random
from collections import deque
import gym

/Users/mxagar/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
from tensorflow.keras.models import Sequential,clone_model
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
env_name = ''